In [2]:
import pandas as pd

import mysql
import mysql.connector

In [4]:
import plotly.express as px
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns



In [6]:
db = mysql.connector.connect(host     = "localhost",
                             user     = "root",
                             password = "contraseña1234",
                             database = "ofertas_empleo")
cursor = db.cursor()

In [8]:
def conectar_db(host = "localhost", user = "root", password = "contraseña1234", database = "ofertas_empleo"):
    return mysql.connector.connect(host = host,
                                    user = user,
                                    password = password,
                                    database = database
                                )

In [10]:
def ejecutar_query(query, db_conectada):
    cursor = db_conectada.cursor()
    cursor.execute(query)
    data = cursor.fetchall()
    columnas = [col[0] for col in cursor.description]
    df = pd.DataFrame(data, columns=columnas)
    cursor.close()
    return df

In [12]:
#cantidad de ofertas por ubicacion
query = '''SELECT 
            u.ubicacion,
            COUNT(o.id_urls) AS cantidad_ofertas
         FROM ofertas o
         JOIN ubicacion u ON o.id_ubicacion = u.id_ubicacion
         GROUP BY u.ubicacion
         ORDER BY cantidad_ofertas DESC;'''

db_conectada = conectar_db()

df_ofertas_por_ubicacion = ejecutar_query(query, db_conectada)

print(df_ofertas_por_ubicacion)

db_conectada.close()

           ubicacion  cantidad_ofertas
0             Madrid              1629
1        Teletrabajo               813
2          Barcelona               591
3             Málaga               206
4           Valencia               115
..               ...               ...
86         Santander                 1
87              Lugo                 1
88     San Sebastián                 1
89    Madrid, Málaga                 1
90  Palma, Barcelona                 1

[91 rows x 2 columns]


In [14]:
#media salarial por función en cada ubicacion
query = '''SELECT 
            u.ubicacion,
            o.funcion,
            AVG((o.salario_min + o.salario_max)/2) AS salario_promedio
        FROM ofertas o
        JOIN ubicacion u ON o.id_ubicacion = u.id_ubicacion
        WHERE o.salario_min IS NOT NULL AND o.salario_max IS NOT NULL
        GROUP BY u.ubicacion, o.funcion
        ORDER BY salario_promedio DESC;'''

db_conectada = conectar_db()

df_salario_por_funcion_ubicacion = ejecutar_query(query, db_conectada)

print(df_salario_por_funcion_ubicacion)

db_conectada.close()

       ubicacion           funcion  salario_promedio
0       Alemania         Consultor          113000.0
1    Reino Unido    Arquitecto TIC           91000.0
2    Teletrabajo    Jefe de Equipo           90500.0
3         Suecia       Programador           90000.0
4         Madrid  Técnico Software           87000.0
..           ...               ...               ...
219       Málaga   Soporte Técnico           16000.0
220   Valladolid          Operador           15500.0
221     Asturias    Diseño gráfico           15500.0
222    Cantabria          Helpdesk           15500.0
223  Teletrabajo   Soporte Técnico             850.0

[224 rows x 3 columns]


In [16]:
#obtener los salarios medios de las ofertas
query = '''SELECT 
            (salario_min + salario_max) / 2 AS salario_medio
        FROM ofertas
        WHERE salario_min IS NOT NULL AND salario_max IS NOT NULL;'''

db_conectada = conectar_db()

df_salarios_ofertas = ejecutar_query(query, db_conectada)

print(df_salarios_ofertas)

db_conectada.close()

      salario_medio
0           52500.0
1           52500.0
2           86000.0
3           40000.0
4           70000.0
...             ...
2004        28500.0
2005        27000.0
2006        42000.0
2007        40500.0
2008        27000.0

[2009 rows x 1 columns]


In [18]:
#Obtener la media y la desviación estándar de los salarios
query = '''SELECT 
            AVG((salario_min + salario_max)/2) AS media_salario,
            STDDEV((salario_min + salario_max)/2) AS std_salario
        FROM ofertas
        WHERE salario_min IS NOT NULL AND salario_max IS NOT NULL;'''

db_conectada = conectar_db()

df_media_desviacionstd_salarios = ejecutar_query(query, db_conectada)

print(df_media_desviacionstd_salarios)

db_conectada.close()

   media_salario   std_salario
0   46805.893479  15385.761889


In [20]:
#Salario medio por oferta y ubicación
query = '''SELECT 
            u.ubicacion,
            (o.salario_min + o.salario_max) / 2 AS salario_medio
        FROM ofertas o
        JOIN ubicacion u ON o.id_ubicacion = u.id_ubicacion
        WHERE o.salario_min IS NOT NULL AND o.salario_max IS NOT NULL;'''

db_conectada = conectar_db()

df_salario_medio_por_oferta_ubicacion = ejecutar_query(query, db_conectada)

print(df_salario_medio_por_oferta_ubicacion)

db_conectada.close()

            ubicacion  salario_medio
0              Madrid        45000.0
1              Madrid        67500.0
2              Madrid        40000.0
3              Madrid        60000.0
4              Madrid       100000.0
..                ...            ...
990  Madrid, Zaragoza        50000.0
991  Madrid, Zaragoza        41500.0
992     San Sebastián        37500.0
993    Madrid, Málaga        35000.0
994  Palma, Barcelona        57500.0

[995 rows x 2 columns]


In [22]:
#salario promedio solo por ubicación. útil si la gráfico es más comparativo que dispersivo
query = '''SELECT 
            u.ubicacion,
            AVG((o.salario_min + o.salario_max) / 2) AS salario_promedio
        FROM ofertas o
        JOIN ubicacion u ON o.id_ubicacion = u.id_ubicacion
        WHERE o.salario_min IS NOT NULL AND o.salario_max IS NOT NULL
        GROUP BY u.ubicacion
        ORDER BY salario_promedio DESC;'''

db_conectada = conectar_db()

df_salario_medio_ubicacion = ejecutar_query(query, db_conectada)

print(df_salario_medio_ubicacion)

db_conectada.close()

                    ubicacion  salario_promedio
0                      Suecia      86500.000000
1                    Alemania      80785.714286
2                 Reino Unido      76666.666667
3                     Logroño      65000.000000
4   Madrid, Valencia, Sevilla      65000.000000
5                     Holanda      62250.000000
6            Palma, Barcelona      57500.000000
7            A Coruña, Madrid      57500.000000
8                     Navarra      54500.000000
9                     Córdoba      54000.000000
10                    Andorra      54000.000000
11           Zaragoza, Madrid      50000.000000
12                     Málaga      49966.666667
13                     España      48812.500000
14           Valencia, Madrid      48500.000000
15           Madrid, A Coruña      47750.000000
16                      Gijón      47500.000000
17                  Guipúzcoa      47500.000000
18                   Palencia      46500.000000
19                   Albacete      46500

In [24]:
#Salario medio, experiencia y función
query = '''SELECT 
            o.funcion,
            o.experiencia,
            (o.salario_min + o.salario_max) / 2 AS salario_medio
        FROM ofertas o
        WHERE o.salario_min IS NOT NULL 
          AND o.salario_max IS NOT NULL
          AND o.experiencia IS NOT NULL;'''

db_conectada = conectar_db()

df_salario_medio_experiencia_funcion = ejecutar_query(query, db_conectada)

print(df_salario_medio_experiencia_funcion)

db_conectada.close()

               funcion  experiencia  salario_medio
0            Marketing          3.0        42000.0
1             Analista          2.0        57000.0
2             Big Data          3.0        57000.0
3        Administrador         10.0        52500.0
4        Administrador          0.0        42000.0
..                 ...          ...            ...
725           Analista          1.0        28500.0
726             Tester          2.0        27000.0
727          Consultor          3.0        42000.0
728  Desarrollador Web          3.0        40500.0
729     Diseño gráfico          2.0        27000.0

[730 rows x 3 columns]


In [26]:
#Opcional: agrupar el promedio por experiencia y función
query = '''SELECT 
            o.funcion,
            o.experiencia,
            AVG((o.salario_min + o.salario_max) / 2) AS salario_promedio
        FROM ofertas o
        WHERE o.salario_min IS NOT NULL 
          AND o.salario_max IS NOT NULL
          AND o.experiencia IS NOT NULL
        GROUP BY o.funcion, o.experiencia
        ORDER BY o.funcion, o.experiencia;'''

db_conectada = conectar_db()

df_salario_promedio_experiencia_funcion = ejecutar_query(query, db_conectada)

print(df_salario_promedio_experiencia_funcion)

db_conectada.close()

              funcion  experiencia  salario_promedio
0       Administrador          0.0      42000.000000
1       Administrador          1.0      33000.000000
2       Administrador          2.0      38000.000000
3       Administrador          3.0      37275.000000
4       Administrador          5.0      44250.000000
..                ...          ...               ...
119  Técnico Hardware          3.0      36000.000000
120  Técnico Software          1.0      55125.000000
121            Tester          1.0      29500.000000
122            Tester          2.0      39166.666667
123            Tester          3.0      36300.000000

[124 rows x 3 columns]


In [38]:
#Skills más solicitadas por funcion
query = '''SELECT 
          o.funcion,
          s.skills        AS skill,
          COUNT(*)        AS veces
        FROM ofertas o
        JOIN oferta_skill os  ON o.id_urls    = os.id_urls
        JOIN skills        s  ON os.id_skills = s.id_skills
        GROUP BY o.funcion, s.skills
        ORDER BY o.funcion, veces DESC;'''

db_conectada = conectar_db()

df_skill_mas_solicitados_por_puesto = ejecutar_query(query, db_conectada)

print(df_skill_mas_solicitados_por_puesto)

db_conectada.close()

   funcion                          skill  veces
0     None        Adaptabilidad al cambio    267
1     None           Aprendizaje Continuo    177
2     None              Trabajo en equipo    134
3     None            Atención al detalle    116
4     None       Capacidad de autogestión     99
5     None                   Proactividad     67
6     None            Comunicación verbal     57
7     None                      Liderazgo     54
8     None    Autonomía en el aprendizaje     43
9     None           Comunicación escrita     33
10    None          Pensamiento analítico     33
11    None              Gestión de equipo     21
12    None      Capacidad de dar feedback     15
13    None     Habilidades de negociación     10
14    None           Pensamiento creativo      7
15    None     Comunicación intercultural      6
16    None           Capacidad de escucha      6
17    None  Capacidad de recibir feedback      5
18    None         Inteligencia emocional      4
19    None     Capac

In [28]:
#Tecnologías más solicitadas por funcion
query = '''SELECT
            o.funcion,
            t.tecnologias_aptitudes AS habilidad_tecnica,
            COUNT(*) AS cantidad_ofertas
        FROM
            ofertas o
        JOIN
            oferta_tecnologia ot ON o.id_urls = ot.id_urls
        JOIN
            tecnologias_aptitudes t ON ot.id_tecnologias = t.id_tecnologias
        GROUP BY
            o.funcion, t.tecnologias_aptitudes
        ORDER BY
            o.funcion, cantidad_ofertas DESC;'''

db_conectada = conectar_db()

df_tecnologias_mas_solicitadas_por_puesto = ejecutar_query(query, db_conectada)

print(df_tecnologias_mas_solicitadas_por_puesto)

db_conectada.close()

     funcion habilidad_tecnica  cantidad_ofertas
0       None              Java               139
1       None        JavaScript               102
2       None            Python               101
3       None               PHP                92
4       None               AWS                49
...      ...               ...               ...
1276  Tester           Vsphere                 1
1277  Tester           Postman                 1
1278  Tester              apis                 1
1279  Tester               ALM                 1
1280  Tester             VoLTE                 1

[1281 rows x 3 columns]
